# **Fine-tuning du modèle Microsoft Phi-3 pour la fourniture de conseils en santé mentale**


#### Problématique
L'amélioration des modèles de langage pour fournir des conseils en santé mentale est d'une importance capitale. En effet, de nombreuses personnes recherchent des conseils et du soutien en ligne pour faire face à des problèmes de santé mentale. Un modèle capable de générer des réponses précises, empathiques et utiles peut non seulement améliorer l'accès aux ressources de santé mentale, mais aussi alléger la charge des professionnels en offrant des réponses initiales et en orientant les utilisateurs vers des solutions adaptées. Le fine-tuning du modèle Microsoft Phi-3 sur un dataset spécialisé permettrait de renforcer ses capacités à fournir des réponses pertinentes et bienveillantes dans ce domaine sensible.

#### Dataset
Le dataset "Amod/mental_health_counseling_conversations" est une collection de questions et de réponses issues de deux plateformes en ligne de counseling et de thérapie. Les questions couvrent une large gamme de sujets liés à la santé mentale, et les réponses sont fournies par des psychologues qualifiés.

- **Context** : Il s'agit de la question posée par un utilisateur, représentant les préoccupations ou les problèmes spécifiques liés à la santé mentale.
- **Response** : Il s'agit de la réponse fournie par un psychologue, contenant des conseils et des orientations pour aider l'utilisateur.


Ce dataset est idéal pour le fine-tuning du modèle Microsoft Phi-3 dans le but d'améliorer ses capacités à générer des conseils en santé mentale, en combinant précision, pertinence et empathie dans ses réponses.

# Bibliothèques & Installations 🗂¶

Imports importants expliqués et exemple d'utilisation basique :

#### datasets
- **But** : bibliothèque pour charger et traiter facilement les ensembles de données de HF.
- **Utilisation Basique** :
  ```python
  from datasets import load_dataset, load_from_disk
  dataset = load_dataset('path/to/dataset', split='train')
  ```

#### peft
- **But** : fournit des utilitaires pour un ajustement fin efficace des paramètres.
- **Utilisation Basique** :
  ```python
  from peft import LoraConfig, prepare_model_for_kbit_training
  lora_config = LoraConfig()
  ```

#### transformers
- **But** : fournit des classes et des fonctions pour les modèles transformers.
- **Utilisation Basique** :
  ```python
  from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, HfArgumentParser, TrainingArguments
  model = AutoModelForCausalLM.from_pretrained('model_name')
  tokenizer = AutoTokenizer.from_pretrained('model_name')
  ```

#### trl
- **But** : fournit des utilitaires pour entraîner des modèles transformers avec l'apprentissage par renforcement (mais pas que).
- **Utilisation Basique** :
  ```python
  from trl import SFTTrainer
  trainer = SFTTrainer(model, tokenizer)
  ```

In [1]:
!pip install -q torch peft bitsandbytes scipy trl transformers accelerate einops tqdm huggingface_hub --use-deprecated=legacy-resolver

ERROR: pip's legacy dependency resolver does not consider dependency conflicts when selecting packages. This behaviour is the source of the following dependency conflicts.
kfp 2.5.0 requires google-cloud-storage<3,>=2.2.1, but you'll have google-cloud-storage 1.44.0 which is incompatible.


In [2]:
import os
import torch
import pandas as pd
from datasets import load_dataset, load_from_disk
from peft import LoraConfig, prepare_model_for_kbit_training
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, HfArgumentParser, TrainingArguments 
from trl import SFTTrainer
from huggingface_hub import notebook_login
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split

2024-07-07 20:18:14.044959: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-07 20:18:14.045066: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-07 20:18:14.171869: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [6]:
print(f"Version pytorch --> {torch.__version__}")

Version pytorch --> 2.1.2


In [7]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"device --> {device}")

device --> cuda:0


In [8]:
### pour se connecter à Hugging Face
notebook_login()

# Préparation des données 🛢

In [9]:
dataset = load_dataset("Amod/mental_health_counseling_conversations", split="train")
dataset

Generating train split:   0%|          | 0/3512 [00:00<?, ? examples/s]

Dataset({
    features: ['Context', 'Response'],
    num_rows: 3512
})

In [10]:
df = pd.DataFrame(dataset)
df.head()

,Context,Response
0,I'm going through some things with my feelings...,"If everyone thinks you're worthless, then mayb..."
1,I'm going through some things with my feelings...,"Hello, and thank you for your question and see..."
2,I'm going through some things with my feelings...,First thing I'd suggest is getting the sleep y...
3,I'm going through some things with my feelings...,Therapy is essential for those that are feelin...
4,I'm going through some things with my feelings...,I first want to let you know that you are not ...


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3512 entries, 0 to 3511
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Context   3512 non-null   object
 1   Response  3512 non-null   object
dtypes: object(2)
memory usage: 55.0+ KB


In [12]:
def format_row(row):
    question = row["Context"]
    response = row["Response"]
    formatted_string = f"[INST] {question} [/INST] {response}"
    return formatted_string

df["Text"] = df.apply(format_row, axis=1)
df.head(3)

,Context,Response,Text
0,I'm going through some things with my feelings...,"If everyone thinks you're worthless, then mayb...",[INST] I'm going through some things with my f...
1,I'm going through some things with my feelings...,"Hello, and thank you for your question and see...",[INST] I'm going through some things with my f...
2,I'm going through some things with my feelings...,First thing I'd suggest is getting the sleep y...,[INST] I'm going through some things with my f...


In [13]:
new_df = df[["Text"]]
train, test = train_test_split(new_df, test_size=0.2, shuffle=True)

In [14]:
train.to_csv("train_data.csv", index=False)
test.to_csv("test_data.csv", index=False)

In [15]:
train_dataset = load_dataset("csv", data_files="train_data.csv", split="train")
test_dataset  = load_dataset("csv", data_files="test_data.csv", split="train")

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [16]:
train_dataset

Dataset({
    features: ['Text'],
    num_rows: 2809
})

# Fine-tuning 🦾

In [17]:
base_model = "microsoft/phi-2"
new_model  = "phi2-ft-mental-health"

#### Tokenizer

1. **Chargement du Tokenizer** (`AutoTokenizer.from_pretrained`) : charge un tokenizer pré-entraîné pour le modèle spécifié.
     - **Paramètres** :
       - `base_model` : identifiant du modèle pré-entraîné (par exemple, un nom de modèle ou un chemin).
       - `use_fast=True` : spécifie que la version rapide du tokenizer doit être utilisée. Les tokenizers rapides sont généralement plus efficaces et plus rapides.  

2. **Définition du token de padding** (`tokenizer.pad_token = tokenizer.eos_token`) : définit le jeton de remplissage du tokenizer pour qu'il soit le même que le jeton de fin de séquence (EOS). Dans certains modèles et configurations d'entraînement, il est utile d'utiliser le jeton EOS à des fins de remplissage pour garantir la cohérence du processus de tokenisation.

3. **Spécification du Côté de Remplissage** (`tokenizer.padding_side = "right"`) : spécifie de quel côté de la séquence les jetons de remplissage doivent être ajoutés.
   - **Options** :
     - `"right"` : Les jetons de remplissage sont ajoutés du côté droit de la séquence.
     - `"left"` : Les jetons de remplissage sont ajoutés du côté gauche de la séquence.

In [18]:
tokenizer = AutoTokenizer.from_pretrained(base_model, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

tokenizer_config.json:   0%|          | 0.00/7.34k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


#### BitsAndBytesConfig

1. **Classe** : `BitsAndBytesConfig`
2. **Paramètres** :
   - `load_in_4bit=True` : activer ou non le chargement du modèle en précision 4 bits (économise la mémoire et accélère les calculs).
   - `bnb_4bit_quant_type="nf4"` : Spécifie le type de quantification pour la précision 4 bits.
     - **Options** : 
       - `"nf4"` : Quantification Non-Flottante 4 bits, plus efficace en termes de mémoire.
       - `"fp4"` : Quantification en Virgule Flottante 4 bits, offrant une plus grande précision.
   - `bnb_4bit_compute_dtype=torch.float16` : définit le type de données pour les calculs avec une précision de 4 bits.
     - **Options** : 
       - `torch.float16` : Utilise des flottants 16 bits pour des calculs plus rapides avec moins de mémoire utilisée.
       - `torch.float32` : Utilise des flottants 32 bits pour une plus grande précision mais plus de mémoire utilisée.
   - `bnb_4bit_use_double_quant=False` : détermine s'il faut utiliser la double quantification, qui applique la quantification deux fois pour une meilleure précision (`True` : applique la double quantification pour une meilleure précision, `False` : ne pas appliquer).

#### LoraConfig

1. **Classe** : `LoraConfig`
2. **Paramètres** :
   - `r=32` : rang de la matrice de faible rang dans LoRA, contrôlant la capacité de l'adaptation.
     - **Options** : Valeurs entières (par exemple, 4, 8, 16, 32). Des valeurs plus élevées augmentent la capacité mais nécessitent plus de calculs.
   - `lora_alpha=64` : un facteur de mise à l'échelle pour les mises à jour de faible rang, affectant le taux d'apprentissage de l'adaptation.
     - **Options** : Valeurs entières (par exemple, 16, 32, 64, 128). Des valeurs plus élevées peuvent entraîner des mises à jour plus importantes.
   - `lora_dropout=0.05` : le taux de dropout appliqué aux mises à jour de faible rang pour éviter le surajustement.
     - **Options** : Valeurs flottantes entre 0 et 1 (par exemple, 0.1, 0.2, 0.5). Des valeurs plus élevées augmentent la régularisation.
   - `bias_type="none"` : Spécifie comment gérer les termes de biais dans les couches LoRA.
     - **Options** : 
       - `"none"` : aucun terme de biais n'est adapté.
       - `"all"` : tous les termes de biais sont adaptés.
       - `"some"` : seuls certains termes de biais sont adaptés.
   - `task_type="CAUSAL_LM"` : type de tâche pour laquelle LoRA est utilisé.
     - **Options** : 
       - `"CAUSAL_LM"` : Modélisation de Langage Causal, pour des tâches auto-régressives.
       - `"SEQ2SEQ_LM"` : Modélisation de Langage Séquence-à-Séquence, pour des tâches de traduction ou de résumé.
   - `target_modules=["Wqkv", "fc1", "fc2"]` : couches du modèle où LoRA sera appliqué.
     - **Options** : Liste de noms de couches (par exemple, `["Wqkv"]`, `["fc1"]`, `["fc2"]`). Spécifique à l'architecture du modèle à ajuster.

In [21]:
bnb_configs = BitsAndBytesConfig(   load_in_4bit=True,
                                    bnb_4bit_quant_type="nf4",
                                    bnb_4bit_compute_dtype=torch.float16,
                                    bnb_4bit_use_double_quant=True
                                )

peft_configs = LoraConfig(  r=8,
                            lora_alpha=32,
                            lora_dropout=0.1,
                            bias="none",
                            task_type="CAUSAL_LM",
                            target_modules="all-linear"
                         )

## Initialisation et configurations du modèle

1. **Initialisation du Modèle** (`AutoModelForCausalLM.from_pretrained`) : charge un modèle de langage causal pré-entraîné.
   - **Paramètres** :
     - `base_model` : identifiant du modèle pré-entraîné (par exemple, nom ou chemin du modèle).
     - `flash_attn=True` : active le mécanisme d'attention Flash, qui optimise les mécanismes d'attention pour améliorer les performances.
     - `flash_rotary=True` : active le mécanisme Rotary Flash, qui améliore les embeddings rotatifs pour un meilleur apprentissage de la représentation.
     - `fused_dense=True` : utilise des opérations denses fusionnées, combinant plusieurs opérations en un seul noyau pour plus d'efficacité.
     - `low_cpu_mem_usage=True` : optimise l'utilisation de la mémoire CPU, réduisant l'empreinte mémoire lors de l'exécution du modèle.
     - `device_map={"": 0}` : mappe les appareils pour les composants du modèle.
     - `revision="refs/pr/23"` : spécifie une révision spécifique du modèle à charger.

2. **Configurations du Modèle** :
   - `model.config.use_cache = False` : désactive la mise en cache des calculs internes dans le modèle.
   - `model.config.pretraining_tp = 1` : définit un paramètre spécifique de la tâche de pré-entraînement à 1.

3. **Préparation du Modèle pour l'entraînement en k-bit** (`prepare_model_for_kbit_training`) : prépare le modèle pour l'entraînement avec quantification en k-bit.
   - **Paramètres** :
     - `model` : L'instance du modèle à préparer pour l'entraînement.
     - `use_gradient_checkpointing=True` : Active le gradient checkpointing pour l'efficacité de la mémoire pendant l'entraînement.

In [22]:
model = AutoModelForCausalLM.from_pretrained(
                                base_model,
                                quantization_config=bnb_configs,
                                torch_dtype=torch.float16,
                                trust_remote_code=True,
                                flash_attn=True,
                                flash_rotary=True,
                                fused_dense=True,
                                low_cpu_mem_usage=True,
                                device_map=device,
                                revision="refs/pr/23"
                                )

#model.to(device)
model.config.use_cache = False
model.config.pretraining_tp = 1

model = prepare_model_for_kbit_training(model, use_gradient_checkpointing=True)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

You are using an old version of the checkpointing format that is deprecated (We will also silently ignore `gradient_checkpointing_kwargs` in case you passed it).Please update to the new format on your modeling file. To use the new format, you need to completely remove the definition of the method `_set_gradient_checkpointing` in your model.


## Entraînement du modèle 🤖

**Paramètres des arguments d'entraînement** (`TrainingArguments`) :
 - `num_train_epochs=1` : nombre de fois que le modèle sera entraîné sur l'ensemble du jeu de données.
 - `per_device_train_batch_size=2` : nombre d'échantillons d'entraînement traités simultanément sur chaque appareil (GPU ou CPU).
 - `gradient_accumulation_steps=32` : nombre de lots pour accumuler les gradients avant d'effectuer une passe arrière.
   - **Objectif** : aide à l'entraînement avec des tailles de lot effectives plus grandes que la mémoire ne le permet, utile lorsque la mémoire GPU est limitée.
   - **Options** : valeurs entières (par exemple, 1, 2, 4, 8, etc.).
 - `evaluation_strategy="steps"` : détermine quand effectuer une évaluation pendant l'entraînement.
   - **Objectif** : spécifie la stratégie pour évaluer le modèle pendant l'entraînement, basée sur les étapes, les époques ou aucune évaluation.
   - **Options** : `"no"` (pas d'évaluation), `"steps"` (évaluation tous les `eval_steps`), `"epoch"` (évaluation à la fin de chaque époque).
 - `eval_steps=1500` : intervalle en étapes pour l'évaluation si `evaluation_strategy="steps"`.
 - `logging_steps=1500` : intervalle en étapes pour la journalisation des métriques d'entraînement dans la console ou les fichiers.
 - `optim="paged_adamw_8bit"` : type d'optimiseur utilisé pour l'entraînement, ici utilisant **paged AdamW avec une précision de 8 bits**.
   - **Options** : dépend de l'implémentation spécifique et des optimiseurs disponibles.
 - `learning_rate=2e-4` : taux d'apprentissage initial pour l'optimiseur.
   - **Objectif** : contrôle la taille des pas pendant la descente de gradient ou l'optimisation.
   - **Options** : valeurs flottantes (par exemple, 0.001, 0.0001, etc.).
 - `lr_scheduler_type="cosine"` : type de planificateur de taux d'apprentissage appliqué pendant l'entraînement.
   - **Objectif** : ajuste le taux d'apprentissage pendant l'entraînement pour optimiser la convergence du modèle.
   - **Options** : `"linear"`, `"cosine"`, `"step"`, `"polynomial"`, etc., selon l'implémentation du planificateur.
 - `save_steps=1500` : intervalle en étapes pour enregistrer les points de contrôle du modèle.
 - `warmup_ratio=0.05` : ratio du nombre total d'étapes d'entraînement pour lesquelles le taux d'apprentissage sera augmenté progressivement.
   - **Objectif** : empêche le modèle de diverger pendant les premières étapes de l'entraînement en augmentant lentement le taux d'apprentissage.
   - **Options** : valeurs flottantes entre 0 et 1 (par exemple, 0.1, 0.05, etc.).
 - `weight_decay=0.01` : force de la régularisation de la décroissance des poids appliquée aux paramètres du modèle pendant l'optimisation.
   - **Objectif** : aide à prévenir le surapprentissage en pénalisant les poids élevés.
   - **Options** : valeurs flottantes (par exemple, 0.001, 0.01, etc.).
 - `max_steps=-1` : nombre maximum d'étapes d'entraînement ; `-1` indique un nombre illimité d'étapes.
   - **Objectif** : limite le nombre d'itérations que le modèle subira pendant l'entraînement.
   - **Options** : valeurs entières ou `-1` pour un nombre illimité d'étapes d'entraînement.

In [23]:
training_args = TrainingArguments(output_dir="trained_weigths",                          ### répertoire pour sauvegarder et identifiant du référentiel
                                num_train_epochs=1,                       ### nombre d'époques d'entraînement
                                per_device_train_batch_size=1,            ### taille du batch par périphérique pendant l'entraînement
                                gradient_accumulation_steps=4,            ### nombre d'étapes avant d'effectuer une passe de rétropropagation/mise à jour
                                gradient_checkpointing=True,              ### utilise le point de contrôle de gradient pour économiser de la mémoire
                                optim="paged_adamw_32bit",
                                save_steps=0,
                                logging_steps=25,                         ### enregistre chaque 25 étapes
                                learning_rate=2e-4,                       ### taux d'apprentissage, basé sur le papier QLoRA
                                weight_decay=0.001,
                                fp16=True,
                                bf16=False,
                                max_grad_norm=0.3,                        ### norme maximale du gradient basée sur le papier QLoRA
                                max_steps=-1,
                                warmup_ratio=0.03,                        ### ratio de préchauffage basé sur le papier QLoRA
                                group_by_length=True,
                                lr_scheduler_type="cosine",               ### utilise le programmeur de taux d'apprentissage cosinus
                                evaluation_strategy="steps"               ### sauvegarde le checkpoint à chaque époque
                                 ) 


trainer = SFTTrainer(   model=model,
                        train_dataset=train_dataset,
                        eval_dataset=test_dataset,
                        peft_config=peft_configs,
                        dataset_text_field="Text",
                        max_seq_length=690,
                        tokenizer=tokenizer,
                        args=training_args
                    )

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1965: FutureWarning: `--push_to_hub_token` is deprecated and will be removed in version 5 of 🤗 Transfor

Map:   0%|          | 0/2809 [00:00<?, ? examples/s]

Map:   0%|          | 0/703 [00:00<?, ? examples/s]

In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss,Validation Loss


# Push to hub

In [ ]:
model.save_pretrained("./trained_model")
model.push_to_hub("anyantudre/phi-2-ft-mental_health")